In [1]:
# 넷플릭스 영화 추천 시스템 구현
import pandas as pd
import numpy as np
from math import sqrt

In [2]:
# 데이터 읽어오기
# 1205 현재 위치 -> 2개 csv 존재, ex07.py
movies = pd.read_csv("./movies.csv")
ratings = pd.read_csv("./ratings.csv")

In [3]:
# 아이템 기반 협업 필터링 구현
data = pd.merge(ratings, movies, on="movieId")
column = ['userId', 'movieId', 'rating', 'title', 'genres']
data = data[column]

moviedata = data.pivot_table(index="movieId", columns='userId')['rating']

C:\Users\user\AppData\Local\Temp\ipykernel_35716\2029323216.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  moviedata = data.pivot_table(index="movieId", columns='userId')['rating']


In [4]:
# NaN 값을 -1 로 변경
moviedata.fillna(-1, inplace=True)

In [5]:
def sim_distance(data, n1, n2):
    # kdd 유사도 함수
    sum = 0
    # 두 사용자가 모두 본 영화를 기준
    for i in data.loc[n1, data.loc[n1, :] >= 0].index:
        if data.loc[n2, i] >= 0:
            # 누적합
            sum += pow(data.loc[n1, i]-data.loc[n2, i], 2)
    return sqrt(1/(sum+1))  # 유사도 형식으로 출력

In [6]:
def top_match(data, name, rank=5, simf=sim_distance):
    # 나와 유사도 가 높은 유저 매칭
    simList = []

    for i in data.index[-10:]:
        if name != i:
            simList.append((simf(data, name, i), i))

    simList.sort()
    simList.reverse()
    return simList[:rank]

In [7]:
def recommendation(data, person, simf=sim_distance):
    # 추천 시스템 함수
    result_top = top_match(data, person, len(data))
    score_dic = {}
    sim_dic = {}
    my_list = []
    for sim, name in result_top:
        if sim < 0:
            continue
        for movie in data.loc[person, data.loc[person, :] < 0].index:
            sim_sum = 0
            if data.loc[name, movie] >= 0:
                sim_sum += data.loc[name, movie]

                score_dic.setdefault(movie, 0)
                score_dic[movie] += sim_sum

                sim_dic.setdefault(movie, 0)
                sim_dic[movie] += sim

    for key in score_dic:
        my_list.append((score_dic[key]/sim_dic[key], key))
    my_list.sort()
    my_list.reverse()
    return my_list

In [8]:
# 20번 user가 안본 영화중에서 추천 점수가 가장 높은 순으로 예상 평점과 영화 제목을 추천(15개까지)
movie_list = []
for rate, m_id in recommendation(moviedata, 20):
    movie_list.append((rate, movies.loc[movies['movieId'] == m_id,
                                        'title'].values[0]))
    if len(movie_list) == 15:
        break

print(movie_list[:15])

[(4.5, "Dracula (Bram Stoker's Dracula) (1992)"), (4.0, 'Entity, The (1981)'), (3.5, 'Born Yesterday (1993)'), (3.5, 'Great Muppet Caper, The (1981)'), (3.5, 'Not One Less (Yi ge dou bu neng shao) (1999)'), (3.5, 'Return with Honor (1998)'), (3.0, 'Left Behind II: Tribulation Force (2002)'), (2.5, 'Bully (2001)'), (2.0, 'Cecil B. DeMented (2000)'), (1.0, "Christmas Vacation (National Lampoon's Christmas Vacation) (1989)"), (0.5, 'Here on Earth (2000)')]
